In [1]:
## Pascal VOC2012 语义分割数据集

%matplotlib inline
import os
import torch
import torchvision
from d2l import torch as d2l

ModuleNotFoundError: No module named 'torchvision'

In [ ]:
# @save
# d2l.DATA_HUB['voc2012'] = (d2l.DATA_URL + 'VOCtrainval_11-May-2012.tar',
#                           '4e443f8a2eca6b1dac8a6c57641b67dd40621a49')

# voc_dir = d2l.download_extract('voc2012', 'VOCdevkit/VOC2012')

voc_dir = r"E:\郭松-深度学习\VOCdevkit\VOC2012"

: 

https://blog.csdn.net/weixin_43917574/article/details/118571388
可以借鉴的一些导入数据集的方法

In [ ]:
#@save
## 将所有输入的图像和标签读入内存
# 为了使用batch训练，需要将所有图片固定一个输入尺寸。
# 由于语义分割对像素精度要求很高，而标签文件也是图片，
# 直接resize会造成误差，所以只能使用随机裁剪成固定尺寸：

def read_voc_images(voc_dir, is_train=True): # = ture 训练数据集
    # = false 验证数据集
    """读取所有VOC图像并标注"""
    txt_fname = os.path.join(voc_dir, 'ImageSets', 'Segmentation',
                             'train.txt' if is_train else 'val.txt')
    # train.txt 哪些训练 val.txt 哪些验证集
    # os.path.join()函数用于路径拼接文件路径，可以传入多个路径

    mode = torchvision.io.image.ImageReadMode.RGB # 彩色
    with open(txt_fname, 'r') as f:
        images = f.read().split()

    features, labels = [], []
    for i, fname in enumerate(images):
    # enumerate() 是迭代器/数组
        features.append(torchvision.io.read_image
                        (os.path.join
                         (voc_dir, 'JPEGImages', f'{fname}.jpg'
            )))
        labels.append(torchvision.io.read_image(os.path.join(
            voc_dir, 'SegmentationClass' ,f'{fname}.png'), mode))
        # 语义分割 - 对每一个像素有label 
        # 这个要存png而不是jpg 为了不压缩 

        # 训练是一张图片 标号也是一张图片 
    return features, labels

train_features, train_labels = read_voc_images(voc_dir, True)

: 

In [ ]:
## 可视化
# 绘制前5个输入图像极其标签

n = 5
imgs = train_features[0:n] + train_labels[0:n] #一行图 一行标签
imgs = [img.permute(1,2,0) for img in imgs]
# permute 将tensor的维度换位
d2l.show_images(imgs, 2, n);
# 可以从图中看到线条 白色的线 背景是黑色的

: 

permute的中文意思是，“排列组合”。permute()可以对某个张量的任意维度进行调换。

import torch
a = torch.randn(2,3,5)
b = a.permute(1,2,0)
print(b.shape)

permute里的参数对应的是张量a的维度索引。所以，permute的输入参数的维度必须与a一致，并且只能是0,1,2...,dim这种数值，这样才能一一对应地索引到a里面的维度。

a.permute(1,2,0)的意思是，把a的第一个维度放到最后面。

结论：
1. 对于同一个tensor，它的元素总数是固定的，permute的作用是把m×n×c，改成n×m×c或c×n×m等任意组合。
2. 对于二维tensor，permute(1,0)做的就是转置，等价于transpose()。


In [ ]:
# 列举rgb颜色值和类名
# 把标签图片转化为对应的标签矩阵

#@save
VOC_COLORMAP = [[0, 0, 0], [128, 0, 0], [0, 128, 0], [128, 128, 0],
                [0, 0, 128], [128, 0, 128], [0, 128, 128], [128, 128, 128],
                [64, 0, 0], [192, 0, 0], [64, 128, 0], [192, 128, 0],
                [64, 0, 128], [192, 0, 128], [64, 128, 128], [192, 128, 128],
                [0, 64, 0], [128, 64, 0], [0, 192, 0], [128, 192, 0],
                [0, 64, 128]]

#@save
VOC_CLASSES = ['background', 'aeroplane', 'bicycle', 'bird', 'boat',
               'bottle', 'bus', 'car', 'cat', 'chair', 'cow',
               'diningtable', 'dog', 'horse', 'motorbike', 'person',
               'potted plant', 'sheep', 'sofa', 'train', 'tv/monitor']

: 

In [ ]:
#@save
# 建立一个索引，将标签图片中每个像素的RGB值一对一映射到对应的类别索引：
# RGB数值与标号互换 

def voc_colormap2label():
    """构建从RGB到VOC类别索引的映射"""
    colormap2label = torch.zeros(256 ** 3, dtype=torch.long) # 字典
    for i, colormap in enumerate(VOC_COLORMAP):
        colormap2label[
            (colormap[0] * 256 + colormap[1]) * 256 + colormap[2]] = i
    return colormap2label
# 一张图片几万个像素 很慢 所以用RGB换算一下 * 256换成整型
#  把tensor换成i
#  = i个RGB的数值映射到i

#@save
def voc_label_indices(colormap, colormap2label):# colormap 图片

    """将VOC标签中的RGB值映射到它们的类别索引"""
    colormap = colormap.permute(1, 2, 0).numpy().astype('int32')
    idx = ((colormap[:, :, 0] * 256 + colormap[:, :, 1]) * 256
           + colormap[:, :, 2]) # 换算成下标
    return colormap2label[idx]

: 

In [ ]:
# 举例：查找第一个图片的标号
y = voc_label_indices(train_labels[0], # 图片
                      voc_colormap2label() # 标号
                      )
# 画飞机的一部分
y[105:115, 130:140], VOC_CLASSES[1]

: 

In [ ]:
## 预处理数据- 图片增广
# 使用图像增广中的随即剪裁，剪裁输入图像和标签的相同区域
# 目标检测不能随便裁剪 - 标号也得裁剪 对起来才行

#@save
def voc_rand_crop(feature, label, height, width):
    """随机裁剪特征和标签图像"""
    # 图片弄出来
    rect = torchvision.transforms.RandomCrop.get_params(
        feature, # 裁剪的是谁
        (height, width) # 高和宽
        )
    feature = torchvision.transforms.functional.crop(feature, *rect)
    label = torchvision.transforms.functional.crop(label, *rect)
    return feature, label

# 看看裁剪是啥效果
# 对飞机图片做五次randomcrop
imgs = []
for _ in range(n):
    imgs += voc_rand_crop(train_features[0], train_labels[0], 200, 300)

# 画出来看看
imgs = [img.permute(1, 2, 0) for img in imgs]
d2l.show_images(imgs[::2] + imgs[1::2], 2, n);

: 

In [ ]:
## 自定义语义分割和数据集

#@save
class VOCSegDataset(torch.utils.data.Dataset):
    """一个用于加载VOC数据集的自定义数据集"""

    def __init__(self, is_train, crop_size, voc_dir):
    # 为什么crop_size 小批量训练 图片一样大才行
    # voc数据集图片不一样大 resize没法拉伸label
        self.transform = torchvision.transforms.Normalize(
            mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
            # 从imagenet里面拿走的
            )
        self.crop_size = crop_size
        # 读近来 不大
        features, labels = read_voc_images(voc_dir, is_train=is_train)
        # 小图片 不要了
        self.features = [self.normalize_image(feature)
                         for feature in self.filter(features)]
        self.labels = self.filter(labels)
        #构造这个函数有开销 所以在Init里面构造
        self.colormap2label = voc_colormap2label()
        print('read ' + str(len(self.features)) + ' examples')

    def normalize_image(self, img):
        return self.transform(img.float() / 255)

    def filter(self, imgs):
        return [img for img in imgs if (
            img.shape[1] >= self.crop_size[0] and
            img.shape[2] >= self.crop_size[1])]
    # 如果图片比设定的crop_Size小 不要了 也可以padding

    def __getitem__(self, idx):
        feature, label = voc_rand_crop(self.features[idx], self.labels[idx],
                                       *self.crop_size)
        # 随机剪裁
        return (feature, voc_label_indices(label, self.colormap2label))
    # 标号RGB换成坐标

    def __len__(self):
        return len(self.features)


: 

In [ ]:
# 读取数据集

crop_size = (320, 480)
voc_train = VOCSegDataset(True, crop_size, voc_dir)
voc_test = VOCSegDataset(False, crop_size, voc_dir)

: 

In [ ]:
batch_size = 1
train_iter = torch.utils.data.DataLoader(voc_train, batch_size, shuffle=True,
                                    drop_last=True,
                                    num_workers=d2l.get_dataloader_workers())
for X, Y in train_iter:
    print(X.shape)
    print(Y.shape)
    break

: 

In [ ]:
# 整合所有组件

#@save
def load_data_voc(batch_size, crop_size):
    """加载VOC语义分割数据集"""
    voc_dir = d2l.download_extract
    ('voc2012', os.path.join(
        'VOCdevkit', 'VOC2012'))
    num_workers = d2l.get_dataloader_workers()
    train_iter = torch.utils.data.DataLoader(
        VOCSegDataset(True, crop_size, voc_dir), batch_size,
        shuffle=True, drop_last=True, num_workers=num_workers)
    test_iter = torch.utils.data.DataLoader(
        VOCSegDataset(False, crop_size, voc_dir), batch_size,
        drop_last=True, num_workers=num_workers)
    return train_iter, test_iter

: 